# 多旅行商问题

# 遗传算法


In [ ]:
from math import floor
from typing import TypeAlias, Any
import numpy as np
from numpy import float64, int64
import numpy.typing as npt
import matplotlib.pyplot as plt  # 导入所需要的库

In [ ]:
ArrayF: TypeAlias = npt.NDArray[float64]
ArrayI: TypeAlias = npt.NDArray[int64]

plt.style.use("seaborn-v0_8")

In [ ]:
class Gena_TSP:
    def __init__(
        self,
        data: ArrayF,
        maxgen: int = 200,
        size_pop: int = 500,
        group: int = 4,
        min_seg_len: int = 3,
        cross_prob: float = 0.9,
        pmuta_prob: float = 0.01,
        select_prob: float = 0.8,
    ) -> None:
        self.maxgen: int = maxgen  # 最大迭代次数
        self.size_pop: int = size_pop  # 群体个数
        self.group: int = group # 多组的数量
        self.min_seg_len: int = min_seg_len # 染色体分段最小长度，即每个旅行商最少经过的城市数
        self.cross_prob: float = cross_prob  # 交叉概率
        self.pmuta_prob: float = pmuta_prob  # 变异概率
        self.select_prob: float = select_prob  # 选择概率

        self.data: ArrayF = data  # 城市的坐标数据
        self.num: int = len(data)  # 城市个数 对应染色体长度
        # 城市间距离的矩阵
        self.matrix_distance: ArrayF = self.matrix_dis()
        # 距离矩阵n*n, 第[i,j]个元素表示城市i到j距离matrix_dis函数见下文

        self.select_num: int = max(floor(self.size_pop * self.select_prob + 0.5), 2)
        # 通过选择概率确定子代的选择个数

        self.chrom: ArrayI = np.array([0] * self.size_pop * self.num).reshape(
            self.size_pop, self.num
        )
        self.sub_sel: ArrayI = np.array([0] * self.select_num * self.num).reshape(
            self.select_num, self.num
        )
        # 父代和子代群体的初始化（不直接用np.zeros是为了保证单个染色体的编码为整数，np.zeros对应的数据类型为浮点型）
        # 可以考虑用np.zeros创建矩阵，再用astype(int)转换为整数型
        # self.chrom: ArrayI = np.zeros((self.size_pop, self.num)).astype(
        #     int64
        # )
        # self.sub_sel: ArrayI = np.zeros((self.size_pop, self.num)).astype(
        #     int64
        # )

        # 分割
        self.segs_chrom: list[list[ArrayI]] = []

        self.fitness: ArrayF = np.zeros(self.size_pop)
        # self.fitness_std: ArrayF = np.zeros(self.size_pop)
        # self.fitness: ArrayF = np.zeros((self.size_pop, self.group))
        # 存储群体中每个染色体的路径总长度，对应单个染色体的适应度就是其倒数
        self.best_fit: list[Any] = []
        self.best_path: list[Any] = []
        # 保存每一步的群体的最优路径和距离

    def matrix_dis(self) -> ArrayF:
        """
        计算城市间的距离函数，得到一个矩阵
        """
        res: ArrayF = np.zeros((self.num, self.num))  # 初始化各点距离矩阵，默认为0.0
        for i in range(self.num):  # 循环选择每个点
            for j in range(i + 1, self.num):  # 再循环选择之后的点
                res[i, j] = np.linalg.norm(
                    self.data[i, :] - self.data[j, :]
                )  # 计算两个点之间的距离，并写入到相应点
                res[j, i] = res[i, j]  # 写入到斜对称的点
        return res

    def rand_chrom(self) -> None:
        """
        随机产生初始化群体函数
        """
        rand_ch: ArrayI = np.array(range(self.num))  # 生成一条初始染色体，对应初始的城市排序
        for i in range(self.size_pop):  # 循环生成size_pop条染色体
            np.random.shuffle(rand_ch)  # 打乱初始排序
            self.chrom[i, :] = rand_ch  # 写入相应位置
            self.fitness[i] = self.comp_fit(rand_ch)  # 将每一条染色体的路径距离值写入fitness矩阵


    def get_segs_path_lens(self, break_points: ArrayI) -> list[int]:
        breakpoints_list: list[int] = break_points.tolist()
        breakpoints_list.append(0)
        breakpoints_list.append(self.num)
        breakpoints_list.sort()
        seg_paths_len: list[int] = [breakpoints_list[i + 1] - breakpoints_list[i] for i in range(0, len(breakpoints_list)-1)]
        return seg_paths_len

    def rand_segs_chrom(self, uniform: bool = True) -> None:
        """
        随机产生初始化群体函数，每条染色体分成多组
        """
        rand_ch: ArrayI = np.array(range(self.num))  # 生成一条初始染色体，对应初始的城市排序
        for i in range(self.size_pop):
            rand_ch_copy: ArrayI = rand_ch.copy()
            # 随机排列染色体上的基因
            np.random.shuffle(rand_ch_copy)
            self.chrom[i, :] = rand_ch_copy  # 写入相应位置
            choice_size: int = self.group - 1 # 分切点个数
            choice_range: ArrayI = np.arange(start=1, stop=self.num - 1, dtype=int64) # type: ignore 分切点取值范围

            if uniform:
                # 染色体分切点，均匀分布
                while True:
                    break_points: ArrayI = np.random.randint(low=1, high=self.num - 1, size=choice_size, dtype=int64) # type: ignore
                    seg_lnes_list: list[int] = self.get_segs_path_lens(break_points)
                    if min(seg_lnes_list) >= self.min_seg_len: # type: ignore
                        break
            else:
                # 染色体分切点，随机分切
                break_points: ArrayI = np.random.choice(choice_range, size=choice_size, replace=False) # type: ignore

            # 染色体随机分切成固定的段数
            break_points: ArrayI = np.sort(break_points)
            path_segs: list[ArrayI] = np.split(rand_ch_copy, break_points, axis=0)

            # 染色体均匀分切成固定的段数
            # path_segs: list[ArrayI] = np.split(rand_ch_copy, self.group, axis=0)
            # self.segs_chrom[i] = path_segs
            self.segs_chrom.append(path_segs)
            # 将每一条分成多组染色体的路径距离值写入fitness矩阵
            self.fitness[i] = self.multi_comp_fit(path_segs).sum(axis=0)  # type: ignore
            # self.fitness_std[i] = self.multi_comp_fit(path_segs).std(axis=0)  # type: ignore


    def comp_fit(self, one_path: ArrayI) -> float:
        """
        计算单个染色体的路径距离值，可利用该函数更新self.fittness
        """
        # res: ArrayF = np.array(0., dtype=float64)
        res: float = 0.0 # 生成单条染色的路径距离值的总和，默认为0
        for i in range(self.num - 1):  # 循环选择出每一个点，即每一个城市
            # 查询得到相邻两个点之间的距离，并加入总和
            res += float(self.matrix_distance[one_path[i], one_path[i + 1]])
        # 查询得到最后一个点和第一个点之间的距离，并加入总和。可能会删除
        # res += float(self.matrix_distance[one_path[-1], one_path[0]])
        return res
    
    def multi_comp_fit(self, one_path_segs: list[ArrayI]) -> ArrayF:
        """
        计算单挑染色体分成多组之后的路径距离值，可利用该self.fittness
        """
        # res: float = 0.0 # 生成单条染色的路径距离值的总和，默认为0
        res_list: list[float] = []
        for seg in one_path_segs:
            if len(seg) == 1:
                # res += 0.0
                res_list.append(0.0)
            else:
                res: float = 0.0
                for i in range(len(seg) - 1):
                    # res += float(self.matrix_distance[seg[i], seg[i + 1]])
                    res += float(self.matrix_distance[seg[i], seg[i + 1]])
                res_list.append(res)
        return np.array(res_list, dtype=float64)
        

    def out_path(self, one_path: ArrayI) -> str:
        """
        路径可视化函数，注意程序的索引值要比现实减1
        """
        res: str = str(one_path[0] + 1) + "-->"  # 第一个点
        for i in range(1, self.num):  # 加入之后所有点
            res += str(one_path[i] + 1) + "-->"
        # res += str(one_path[0] + 1) + "\n"  # 加入最后一个点和第一个点。可能会删除
        return res
        # print(res)

    def out_path_segs(self, one_path_segs: list[ArrayI]) -> list[str]:
        path_segs_list: list[str] = []
        for seg in one_path_segs:
            if len(seg) == 1:
                res: str = f"{seg[0] + 1}-->"
                path_segs_list.append(res)
            else:
                res: str = str(seg[0] + 1) + "-->"
                for i in range(1, len(seg)):
                    res += str(seg[i] + 1) + "-->"
                path_segs_list.append(res)
        return path_segs_list

    def select_sub(self) -> None:
        """
        子代选取，根据选中概率与对应的适应度函数，采用随机遍历选择方法
        """
        fit: ArrayF = 1 / np.std(self.fitness, axis=0)  # type: ignore 适应度等于路径距离值的倒数。获得适应度矩阵
        cumsum_fit: ArrayF = np.cumsum(fit)  # 获得适应度累加的矩阵
        pick: ArrayF = (
            cumsum_fit[-1]
            / self.select_num
            * (np.random.rand() + np.array(range(self.select_num)))
        )
        # 生成从父代染色体群体中选择select_num数量的选中概率矩阵
        i: int = 0  # i是染色体数量的索引，j是选择出的染色体数量的索引
        j: int = 0  # i是染色体数量的索引，j是选择出的染色体数量的索引
        index: list[int] = []  # 选择出的染色体所在索引的列表
        while i < self.size_pop and j < self.select_num:
            # 遍历所有的染色体。如果当前染色体的累计适应度超过当前选择的概率，
            # 则将当前染色体的索引放入选择染色体列表；否则查看下一条
            if cumsum_fit[i] >= pick[j]:
                index.append(i)
                j += 1
            else:
                i += 1
        self.sub_sel = self.chrom[index, :]  # 将选择出的染色体放置到子代群体

    # 交叉，依概率对子代个体进行交叉操作
    def cross_sub(self) -> None:
        # 根据子代选择数量的奇偶，确定子代群体中参与交叉的染色体的索引
        if self.select_num % 2 == 0:
            num = range(0, self.select_num, 2)
        else:
            num = range(0, self.select_num - 1, 2)
        for i in num:
            if self.select_prob > np.random.rand():
                # 如果选择概率超过某个值，则当前的染色体发生交叉
                self.sub_sel[i, :], self.sub_sel[i + 1, :] = self.intercross(
                    self.sub_sel[i, :], self.sub_sel[i + 1, :]
                )

    def intercross(self, ind_a: ArrayI, ind_b: ArrayI) -> tuple[ArrayI, ArrayI]:  # type: ignore
        # 随机生成两个整数作为交叉的基因索引范围，上限是基因数量
        r1: int = np.random.randint(low=0, high=self.num)  # type: ignore
        r2: int = np.random.randint(low=0, high=self.num)  # type: ignore
        while r1 == r2:
            # 避免两个整数是一样的
            r2: int = np.random.randint(low=0, high=self.num)  # type: ignore
        left: int = min(r1, r2)  # 基因索引左边值
        right: int = max(r1, r2)  # 基因索引左边值
        # 复制两条参与交叉的染色体
        ind_a1: ArrayI = ind_a.copy()
        ind_b1: ArrayI = ind_b.copy()
        for i in range(left, right + 1):
            # 再次复制两条参与交叉的染色体
            ind_a2: ArrayI = ind_a.copy()
            ind_b2: ArrayI = ind_b.copy()
            # 交换两条不同染色体上相同索引的基因
            ind_a[i] = ind_b1[i]
            ind_b[i] = ind_a1[i]
            # 查找两条染色体中与当前索引i上的基因相同的其他基因的索引
            # 旨在避免染色体上出现重复的基因
            x: ArrayI = np.argwhere(ind_a == ind_a[i])
            y: ArrayI = np.argwhere(ind_b == ind_b[i])
            # 如果染色体上出现重复基因
            # 则将非当前索引i之外的其他索引的值，都还原为对应染色体上原来的值
            if len(x) == 2:
                ind_a[x[x != i]] = ind_a2[i]
            if len(y) == 2:
                ind_b[y[y != i]] = ind_b2[i]
            return ind_a, ind_b

    def mutation_sub(self) -> None:
        """变异模块"""
        for i in range(self.select_num):
            if np.random.rand() <= self.pmuta_prob:
                # 随机生成两个整数作为变异的基因索引，上限是基因数量
                r1: int = np.random.randint(low=0, high=self.num)  # type: ignore
                r2: int = np.random.randint(low=0, high=self.num)  # type: ignore
                while r1 == r2:
                    # 避免两个整数是一样的
                    r2: int = np.random.randint(low=0, high=self.num)  # type: ignore
                # 交换当前染色体上两个位置的基因
                self.sub_sel[i, [r1, r2]] = self.sub_sel[i, [r2, r1]]

    def reverse_sub(self) -> None:
        """进化逆转"""
        for i in range(self.select_num):
            # 随机生成两个整数作为交换的基因索引，上限是基因数量
            r1: int = np.random.randint(low=0, high=self.num)  # type: ignore
            r2: int = np.random.randint(low=0, high=self.num)  # type: ignore
            while r1 == r2:
                # 避免两个整数是一样的
                r2: int = np.random.randint(low=0, high=self.num)  # type: ignore
            left: int = min(r1, r2)  # 基因索引左边值
            right: int = max(r1, r2)  # 基因索引右边值
            sel: ArrayI = self.sub_sel[i, :].copy()  # 复制当前染色体
            sel[left : right + 1] = self.sub_sel[i, left : right + 1][::-1]  # 颠倒当前染色体的指定基因范围
            # 如果当前逆转的染色体的适应度超过原来的染色体，则替换为当前染色体
            if self.comp_fit(sel) < self.comp_fit(self.sub_sel[i, :]):
                self.sub_sel[i, :] = sel

    def reins(self) -> None:
        """子代插入父代，得到相同规模的新群体"""
        # 获得倒置的群体中每个染色体的路径总长度的索引矩阵
        index: ArrayI = np.argsort(self.fitness)[::-1]
        # 选择索引矩阵中指定选择数量的索引，并保存为子代
        self.chrom[index[: self.select_num], :] = self.sub_sel

    def draw_path(self, path: ArrayI) -> None:
        ## 绘制路径图
        fig1, ax = plt.subplots()  # type: ignore
        x: ArrayF = self.data[:, 0]
        y: ArrayF = self.data[:, 1]
        ax.scatter(x, y, linewidths=0.1)  # type: ignore
        for i, txt in enumerate(range(1, len(self.data) + 1)):
            ax.annotate(txt, (x[i], y[i]))  # type: ignore
        # res = self.chrom[g]
        res: ArrayI = path
        x0: ArrayF = x[res]
        y0: ArrayF = y[res]
        for i in range(len(self.data) - 1):
            plt.quiver(x0[i], y0[i], x0[i + 1] - x0[i], y0[i + 1] - y0[i], color="r", width=0.002, angles="xy", scale=1, scale_units="xy")  # type: ignore
        # plt.quiver(x0[-1], y0[-1],x0[0]-x0[-1], y0[0]-y0[-1], color='r', width=0.005,angles='xy',scale=1, scale_units='xy') # type: ignore
        # plt.show() # type: ignore

    def draw_seg_pathes(self, seg_pathes: list[ArrayI]) -> None:
        # cmap = plt.cm.get_cmap('hsv', 20) # type: ignore
        ## 绘制路径图
        fig1, ax = plt.subplots()  # type: ignore
        x: ArrayF = self.data[:, 0]
        y: ArrayF = self.data[:, 1]
        ax.scatter(x, y, linewidths=0.1)  # type: ignore
        for i, txt in enumerate(range(1, len(self.data) + 1)):
            ax.annotate(txt, (x[i], y[i]))  # type: ignore
        # res = self.chrom[g]
        for path in seg_pathes:
            # color: tuple[float, float, float] = (np.random.random(), np.random.random(), np.random.random())
            # color_i: int = np.random.randint(20)  # type: ignore
            res: ArrayI = path
            x0: ArrayF = x[res]
            y0: ArrayF = y[res]
            for i in range(len(path) - 1):
                plt.quiver(x0[i], y0[i], x0[i + 1] - x0[i], y0[i + 1] - y0[i], width=0.005, angles="xy", scale=1, scale_units="xy")  # type: ignore


In [ ]:
# 路径坐标为
data: ArrayF = np.array(
    [
        16.47,
        96.10,
        16.47,
        94.44,
        20.09,
        92.54,
        22.39,
        93.37,
        25.23,
        97.24,
        22.00,
        96.05,
        20.47,
        97.02,
        17.20,
        96.29,
        16.30,
        97.38,
        14.05,
        98.12,
        16.53,
        97.38,
        21.52,
        95.59,
        19.41,
        97.13,
        20.09,
        92.55,
    ]
).reshape(14, 2)

In [ ]:
Path_short: Gena_TSP = Gena_TSP(data=data, maxgen=500)  # 根据位置坐标，生成一个遗传算法类
# Path_short.rand_chrom() # 初始化父类
Path_short.rand_segs_chrom(uniform=True)

In [ ]:
print("初始染色体的路径：")
print(Path_short.out_path_segs(Path_short.segs_chrom[0]))

In [ ]:
print(f'初始染色体的路程: {Path_short.fitness[0]}')

In [ ]:
# 循环迭代遗传过程
for i in range(Path_short.maxgen):
    Path_short.select_sub()   # 选择子代
    Path_short.cross_sub()    # 交叉
    Path_short.mutation_sub() # 变异
    Path_short.reverse_sub()  # 进化逆转
    Path_short.reins()        # 子代插入
    # 重新计算新群体的距离值
    for j in range(Path_short.size_pop):
        Path_short.fitness[j] = Path_short.multi_comp_fit(Path_short.segs_chrom[j]).sum(axis=0) # type: ignore
        # Path_short.fitness_std[j] = Path_short.multi_comp_fit(Path_short.segs_chrom[j]).std(axis=0)
    # 每隔四十步显示当前群体的最优路径
    index: int = int(Path_short.fitness.argmin())
    # index: int = int(Path_short.fitness.argmin())
    # if (i + 1) % 40 == 0:
    #     print(f"第{i + 1}步后的最短的路程: {Path_short.fitness[index]}")
    #     print(f'第{i + 1}步后的最优路径:')
    #     print(Path_short.out_path_segs(Path_short.segs_chrom[index])) # 显示每一步的最优路径
    # 存储每一步的最优路径及距离
    Path_short.best_fit.append(Path_short.fitness[index]) # type: ignore
    Path_short.best_path.append(Path_short.segs_chrom[index]) # type: ignore

print("最优路径：")
print(Path_short.out_path_segs(Path_short.best_path[-1]))
print(f"最优路径的路程：{Path_short.best_fit[-1]}")

In [ ]:
Path_short.draw_seg_pathes(Path_short.best_path[-1])